# Example

## Preparation

In [4]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv(".env")

True

In [18]:
from langchain_openai_limiter import ChooseKeyChatOpenAI, LimitAwaitChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import sys
from datetime import datetime

## Chat completion exampls

Here we will have 2 types of examples - full completion and streaming one, each in 2 versions - sync & async.

In [15]:
chat_model = ChatOpenAI(
    model_name="gpt-4-0613",
    streaming=True,
)
chat_model_limit_await = LimitAwaitChatOpenAI(
    chat_openai=chat_model,
    limit_await_timeout=60.0,
    limit_await_sleep=0.1,
)
chat_model_key_choose = ChooseKeyChatOpenAI(
    chat_openai=chat_model_limit_await,
    openai_api_keys=[
        os.environ["OPENAI_API_KEY0"],
        os.environ["OPENAI_API_KEY1"],
    ]
)

In [13]:
history = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
print(chat_model_key_choose.invoke(history).content)

J'aime la programmation.


In [14]:
print((await chat_model_key_choose.ainvoke(history)).content)

J'aime la programmation.


In [19]:
for chunk in chat_model_key_choose.stream(history):
    print(str(datetime.now()), chunk.content)

2023-11-04 10:13:51.651513 
2023-11-04 10:13:51.651513 J
2023-11-04 10:13:51.719640 'a
2023-11-04 10:13:51.774681 ime
2023-11-04 10:13:51.835578  la
2023-11-04 10:13:51.889139  programm
2023-11-04 10:13:51.937523 ation
2023-11-04 10:13:52.157775 .
2023-11-04 10:13:52.395186 


In [20]:
async for chunk in chat_model_key_choose.astream(history):
    print(str(datetime.now()), chunk.content)

2023-11-04 10:14:08.852084 
2023-11-04 10:14:08.852084 J
2023-11-04 10:14:08.932487 'a
2023-11-04 10:14:08.984516 ime
2023-11-04 10:14:09.044604  la
2023-11-04 10:14:09.099822  programm
2023-11-04 10:14:09.157269 ation
2023-11-04 10:14:09.221833 .
2023-11-04 10:14:09.273985 


## Embeddings

Here we will have sync & async examples of embeddings generation

In [21]:
from langchain_openai_limiter import ChooseKeyOpenAIEmbeddings, LimitAwaitOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

In [22]:
embedder_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
)
embedder_model_limit_await = LimitAwaitOpenAIEmbeddings(
    openai_embeddings=embedder_model,
    limit_await_timeout=60.0,
    limit_await_sleep=0.1,
)
embedder_model_key_choose = ChooseKeyOpenAIEmbeddings(
    openai_embeddings=embedder_model_limit_await,
    openai_api_keys=[
        os.environ["OPENAI_API_KEY0"],
        os.environ["OPENAI_API_KEY1"],
    ]
)

In [23]:
docs = embedder_model_key_choose.embed_documents([
    "Markdown is a lightweight markup language",
    "Brainfuck is an esoteric programming language",
])
query = embedder_model_key_choose.embed_query("What is Markdown?")

In [27]:
print(" ".join(map("{0:.2f}".format, docs[0])))
print(" ".join(map("{0:.2f}".format, docs[1])))
print(" ".join(map("{0:.2f}".format, query)))

-0.01 0.03 -0.00 -0.00 0.00 0.00 -0.03 -0.03 -0.01 -0.02 0.03 0.04 -0.04 0.02 -0.01 0.02 0.01 -0.01 0.00 0.00 -0.00 0.02 -0.02 -0.01 -0.02 0.02 0.02 -0.02 0.00 0.01 0.02 -0.02 -0.02 -0.01 -0.01 -0.00 -0.01 -0.03 0.01 -0.02 0.01 0.02 0.01 -0.01 -0.01 0.01 0.03 -0.02 -0.00 0.00 0.03 0.01 0.01 -0.00 -0.00 -0.01 0.01 -0.00 0.01 -0.01 -0.01 -0.00 -0.02 -0.01 0.00 0.00 -0.00 0.03 0.00 0.00 0.02 0.02 -0.00 -0.01 0.02 -0.01 -0.01 0.00 0.02 -0.02 0.01 -0.01 0.01 0.03 0.02 -0.01 0.01 0.03 -0.03 -0.02 0.01 0.03 0.01 0.03 -0.01 0.02 0.00 0.02 0.01 -0.01 -0.01 0.03 -0.01 -0.01 -0.02 0.01 0.03 -0.02 0.02 0.01 -0.01 0.06 -0.00 -0.03 -0.01 -0.00 0.01 -0.03 -0.00 0.01 -0.01 0.03 0.02 -0.03 0.02 0.01 -0.02 -0.01 -0.02 0.00 0.04 0.01 -0.00 0.00 -0.01 0.04 -0.00 0.01 -0.03 -0.01 0.02 0.02 -0.01 -0.01 -0.01 -0.01 0.00 -0.02 0.01 -0.02 0.01 -0.01 -0.01 0.00 -0.01 0.03 0.01 0.02 0.01 -0.01 0.01 0.01 0.01 -0.01 0.02 0.04 0.01 0.00 -0.01 -0.01 0.01 0.03 -0.03 0.03 -0.02 -0.01 0.01 0.03 -0.01 -0.01 -0.04 0.01 0

In [28]:
docs = await embedder_model_key_choose.aembed_documents([
    "Markdown is a lightweight markup language",
    "Brainfuck is an esoteric programming language",
])
query = await embedder_model_key_choose.aembed_query("What is Markdown?")

In [29]:
print(" ".join(map("{0:.2f}".format, docs[0])))
print(" ".join(map("{0:.2f}".format, docs[1])))
print(" ".join(map("{0:.2f}".format, query)))

-0.01 0.03 -0.00 -0.00 0.00 0.00 -0.03 -0.03 -0.01 -0.02 0.03 0.04 -0.04 0.02 -0.01 0.02 0.01 -0.01 0.00 0.00 -0.00 0.02 -0.02 -0.01 -0.02 0.02 0.02 -0.02 0.00 0.01 0.02 -0.02 -0.02 -0.01 -0.01 -0.00 -0.01 -0.03 0.01 -0.02 0.01 0.02 0.01 -0.01 -0.01 0.01 0.03 -0.02 -0.00 0.00 0.03 0.01 0.01 -0.00 -0.00 -0.01 0.01 -0.00 0.01 -0.01 -0.01 -0.00 -0.02 -0.01 0.00 0.00 -0.00 0.03 0.00 0.00 0.02 0.02 -0.00 -0.01 0.02 -0.01 -0.01 0.00 0.02 -0.02 0.01 -0.01 0.01 0.03 0.02 -0.01 0.01 0.03 -0.03 -0.02 0.01 0.03 0.01 0.03 -0.01 0.02 0.00 0.02 0.01 -0.01 -0.01 0.03 -0.01 -0.01 -0.02 0.01 0.03 -0.02 0.02 0.01 -0.01 0.06 -0.00 -0.03 -0.01 -0.00 0.01 -0.03 -0.00 0.01 -0.01 0.03 0.02 -0.03 0.02 0.01 -0.02 -0.01 -0.02 0.00 0.04 0.01 -0.00 0.00 -0.01 0.04 -0.00 0.01 -0.03 -0.01 0.02 0.02 -0.01 -0.01 -0.01 -0.01 0.00 -0.02 0.01 -0.02 0.01 -0.01 -0.01 0.00 -0.01 0.03 0.01 0.02 0.01 -0.01 0.01 0.01 0.01 -0.01 0.02 0.04 0.01 0.00 -0.01 -0.01 0.01 0.03 -0.03 0.03 -0.02 -0.01 0.01 0.03 -0.01 -0.01 -0.04 0.01 0